In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df=pd.read_csv('../input/imdb-extensive-dataset/IMDb movies.csv')

In [ ]:
data=pd.DataFrame()

In [ ]:
c=df['avg_vote'].mean()


In [ ]:
m=df['votes'].quantile(0.60)

In [ ]:
data=df[df['votes']>=m]

In [ ]:
score=[]
v=data['votes'].values
r=data['avg_vote'].values

In [ ]:
def weighted_average(v,r,c,m):
    s=((v*r)/(v+m))+((m*c)/(v+m))
    return s

In [ ]:
for i in range(len(v)):
    score.append(weighted_average(v[i],r[i],c,m))

In [ ]:
data['weighted_score']=score


In [ ]:
data=data.sort_values('weighted_score',ascending=False)

In [ ]:
data.head()

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
data['description']=data['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(data['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

In [ ]:
tfidf.get_feature_names()[5000:5010]


In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [ ]:
cosine_sim.shape

In [ ]:
cosine_sim[1]

In [ ]:
indices = pd.Series(data.index, index=data['original_title']).drop_duplicates()

In [ ]:
indices[:10]

In [ ]:
def recommendation(title,cos=cosine_sim):
    idx=indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return data['original_title'].iloc[movie_indices]

In [ ]:
movie=input('Please enter the movie name:')

In [ ]:
recommendation(movie)